In [2]:
import math
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# required for interactive plotting
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import numpy.polynomial as np_poly

from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML

initialization  
$ \newcommand{\E}[1]{\mathbb{E}\left[#1\right]}$  
$ \newcommand{\V}[1]{\mathbb{V}\left[#1\right]}$
$ \newcommand{\cov}[1]{\text{cov} \sigma\left[#1\right]}$
$ \newcommand{\EXP}[1]{\exp\left(#1\right)}$  
$\newcommand{\P}{\mathbb{P}}$
$\newcommand{\ds}{\displaystyle}$
$\newcommand{\mat}[1]{
\left[
\begin{matrix}
#1
\end{matrix}
\right]
}$
$\newcommand{\commentgray}[1]{\color{gray}{\text{#1}}}$
$\newcommand{\arrthree}[1]{
\begin{array}{rlr}
#1
\end{array}
}
$

$
\newcommand{\sumnN}{\sum_{n=1}^{N}}
\newcommand{\sumkM}{\sum_{k=1}^{M}}
\newcommand{\prodnN}{\prod_{n=1}^{N}}
$
$
\newcommand{\sumset}[1]{\stackrel{\Sigma^{*}}{#1}}
$
$
\newcommand{\chib}{\boldsymbol{\chi}}
$

$\newcommand{\Nl}[3]{\mathcal{N}\left(#1 \mid #2, #3\right)}$
$\newcommand{\Nstdx}{\Nl{\mathbf{x}}{\mathbf{\mu}}{\Sigma}}$
$\newcommand{\ab}{\mathbf{a}}$
$\newcommand{\Ab}{\mathbf{A}}$
$\newcommand{\Abt}{\Ab^T}$
$\newcommand{\bb}{\mathbf{b}}$
$\newcommand{\Bb}{\mathbf{B}}$
$\newcommand{\Cb}{\mathbf{C}}$
$\newcommand{\Db}{\mathbf{D}}$
$\newcommand{\Lb}{\mathbf{L}}$
$\newcommand{\Lbi}{\Lb^{-1}}$
$\newcommand{\mb}{\mathbf{m}}$
$\newcommand{\Mb}{\mathbf{M}}$
$\newcommand{\Rb}{\mathbf{R}}$
$\newcommand{\ub}{\mathbf{u}}$
$\newcommand{\Xb}{\mathbf{X}}$
$\newcommand{\xb}{\mathbf{x}}$
$\newcommand{\xab}{\mathbf{x_a}}$
$\newcommand{\xabt}{\mathbf{x_a}^T}$
$\newcommand{\xbb}{\mathbf{x_b}}$
$\newcommand{\xbbt}{\mathbf{x_b}^T}$
$\newcommand{\yb}{\mathbf{y}}$
$\newcommand{\zb}{\mathbf{z}}$
$\newcommand{\Ub}{\mathbf{U}}$

$\newcommand{\mub}{\pmb{\mu}}$
$\newcommand{\muab}{\pmb{\mu}_a}$
$\newcommand{\mubb}{\pmb{\mu}_b}$
$\newcommand{\saa}{\Sigma_{aa}}$
$\newcommand{\sab}{\Sigma_{ab}}$
$\newcommand{\sba}{\Sigma_{ba}}$
$\newcommand{\sbb}{\Sigma_{bb}}$
$\newcommand{\laa}{\Lambda_{aa}}$
$\newcommand{\laai}{\Lambda_{aa}^{-1}}$
$\newcommand{\lab}{\Lambda_{ab}}$
$\newcommand{\lba}{\Lambda_{ba}}$
$\newcommand{\lbb}{\Lambda_{bb}}$
$\newcommand{\lbbi}{\Lambda_{bb}^{-1}}$
$\newcommand{\li}{\Lambda^{-1}}$
$
\newcommand{\etab}{\pmb{\eta}}
\newcommand{\etat}{\eta^T}
\newcommand{\etabt}{\etab^T}
$

$\newcommand{\multivarcoeff}{\frac{1}{(2\pi)^{D/2}}
\frac{1}{\left| \mathbf{\Sigma}\right|^{1/2}}}$
$\newcommand{\multivarexp}[2]
{
\left\{
 -\frac{1}{2} 
 {#1}^T 
 #2
 {#1}
\right\}
}$
$\newcommand{\multivarexpx}[1]{\multivarexp{#1}{\Sigma^{-1}}}$
$\newcommand{\multivarexpstd}{\multivarexpx{(\xb-\mub)}}$
$\newcommand{\gam}{\operatorname{Gam}}$
$
\newcommand{\sumkMl}{\sum_{k=1}^{M-1}}
\newcommand{\sumjMl}{\sum_{j=1}^{M-1}}
$


* We shall assume that the original graph is an undirected tree or a directed tree or polytree
  * so that the corresponding factor graph has a tree structure.
* We first convert the original graph into a factor graph
  * so that we can deal with both directed and undirected models using the same framework. 
* Our goal is to exploit the structure of the graph to achieve two things
:
  1. to obtain an efficient, exact inference algorithm for finding marginals
  1. sin situations where several marginals are required to allow computations to be shared efficiently.

Factor Graph expression
$$
p(\xb) = \prod_s f_s(\xb_s)
$$

We begin by considering the problem of finding the marginal p(x) for particular variable node x. For the moment, we shall suppose that all of the variables are hidden

Finding the marginal of a variable x
$$
p(x) = \sum_{\xb ~\setminus~ x} p(\xb)
$$

$\xb ~\setminus~ x$ is the set of variables in $\xb$ except x

The idea is to substitute for p(x) using the factor graph expression and then interchange summations and products in order to obtain an efficient algorithm. 

**Definitions**
$$
\arrthree{
\text{ne}(x) &: \text{set of factor nodes which are neighbors of x}
\\
X_s &: \text{set of } \mathbf{all} \text{ variables in the subtree connected to x via }f_s
\\
F_s(x, X_s) &: \text{product of all the factors associated with factor } f_s
}
$$

# Marginal at x

Then
$$
\arrthree{
p(x)
&= \ds \sum_{\xb ~\setminus~ x} ~\prod_{s ~\in ~ne(x)} F_s(x, X_s)
\\
&= \ds \prod_{s ~\in ~ne(x)} \left[ \sum_{X_s} F_s(x, X_s) \right]
\\
&= \ds \prod_{s ~\in ~ne(x)} \mu_{f_s \rightarrow x}(x)
}
$$

This says, that the marginal probability of a node x is given by the product of messages from the factor nodes $f_s$ associated with x.

# Msg from factor to node

$$
\mu_{f_s \rightarrow x}(x) = \sum_{\{X_s\}} F_s(x, X_s)
$$
* $\sum\limits_{\{.\}}$ indicates multiple summations, not just a single one.
* This is the message from the factor node $f_s$ to the variable node x.

# Refine: msg from factor to node

* Say, the factor $f_s$ is *directly* and *immediately* connected to variable nodes $X_{si} = \{x_m\}_{m=1}^{M}$ along with x. Here the additional suffix *i* stands for immediate
* Please note that $X_{si} \subseteq X_s$ (ssince the latter is the set of all the node variables under the factor s)

$$
F_s(x, X_s)
= f_s(x, x_1, \cdots, x_M)
\prod_{m ~\in ~ne(f_s) \setminus x} G_m(x_m, X_{sm})
$$
* Fret not, G would be defined soon. Hold on.

That is,
$$
\arrthree{
\mu_{f_s \rightarrow x}(x)
&=
%% line 1
\overbrace{
\sum\limits_{x_1} \cdots \sum\limits_{x_M}}^{
\begin{matrix}
  \text{summations} \\ \text{for imm nbrs}
\end{matrix}
}
~~ f_s(x, x_1, \cdots, x_M)
\overbrace{
\sum\limits_{\{X_s \setminus X_{si}\}}}^{
\begin{matrix}
  \text{terms in }X_s \\ \text{other than } \\ \text{imm nbrs}
\end{matrix}
}
\left[
  \prod\limits_{m \in ne(f_s) \setminus x}
  G_m(x_m, X_{sm})
\right]
%% line 2
\\ &=
\sum\limits_{\{X_{si}\}}
f_s(x, x_1, \cdots, x_M)
\prod\limits_{m ~\in ~ne(f_s) ~\setminus ~x}
\left[
\sum\limits_{\{X_{sm}\}}
G_m(x_m, X_{sm})
\right]
\\ &=
%% line 3
\sum\limits_{\{X_{si}\}}
f_s(x, x_1, \cdots, x_M)
\prod\limits_{m ~\in ~ne(f_s) ~\setminus ~x}
\mu_{x_m \rightarrow f_s}(x_m)
}
$$

* Here $\{X_{sm}\}$ refers to the variable nodes present in the subtree under the variable node $x_m$, which in turn is associated with the factor node $f_s$
* This says that to evaluate the message sent from the factor node $f_s$ to x along the link connecting them, we have to
  1. take the product of the incoming messages $\mu_{[.] \rightarrow f_s}$
  1. multiply by the factor associated with the factor node $f_s$
  1. marginalize over all the variables immediately connected to the factor node $f_s$

We have therefore introduced two distinct kinds of message,
* those that go from factor nodes to variable nodes denoted $\mu_{f \rightarrow x}(x)$
* those that go from variable nodes to factor nodes denoted $\mu_{x \rightarrow f}(x)$.

It is important to note that a factor node can send a message to a variable node once it has received incoming messages from all other neighbouring variable nodes.

* Say $x_m$ is one of the variable nodes connected to factor node $f_s$
* Let $ne(x_m)$ be the set of factor nodes connected directly to $x_m$

Then
$$
G_m(x_m, X_{sm})
=
\prod\limits_{l ~\in ~ne(x_m) ~\setminus ~f_s} F_l(x_m, X_{ml})
$$

* The factors $F_l(x_m, X_{ml})$ denote the subtree of the original graph we began with.
* Also, the form of the equation for G(.) is the same as that of p(x)

Thus,  
$$
\arrthree{
\mu_{x_m \rightarrow f_s}(x_m)
&=
\sum\limits_{\{X_{sm}\}}
G_m(x_m, X_{sm})
&
\commentgray{$\{X_{sm}\}$: 
$\begin{matrix}
    \text{variable nodes under} \\ \text{the subtree rooted at } x_m
\end{matrix}$
}
\\ &=
\sum\limits_{\{X_{sm}\}}
\left[
  \prod\limits_{\ell ~\in ~ne(x_m) ~\setminus ~f_s} F_l(x_m, X_{ml})
\right]
&
\commentgray{$\ell$: $\begin{matrix}
\text{set of factors} \\ 
\text{under } x_m \text{excluding } f_s
\end{matrix}$}
\\ &=
\prod\limits_{l ~\in ~ne(x_m) ~\setminus ~f_s}
\left[
  \sum\limits_{\{X_{ml}\}}
  F_l(x_m, X_{ml})
\right]
&
\commentgray{$\{X_{ml}\}$: $
\begin{matrix}
    \text{set of variable nodes} \\ \text{under the factor }\ell
\end{matrix} 
$}
\\ &=
\prod\limits_{l ~\in ~ne(x_m) ~\setminus ~f_s}
\mu_{f_l \rightarrow x_m} (x_m)
}
$$

* that is, to evaluate the message sent by a variable node to an adjacent factor node, take the product of the incoming messages across all other links.
* Any variable node that has only two neighbours performs no computation but simply passes messages through unchanged.
* A variable node can send a message to a factor node once it has received incoming messages from all other neighbouring factor nodes.

# Putting it all together

* Our original goal was to calculate the marginal for variable node x
* This marginal is given by the product of incoming messages along all of the links arriving at that node.
  * Each of these messages can be computed recursively in terms of other messages.

* In order to start this recursion, we can view the node x as the root of the tree and begin at the leaf nodes.
* From the definition of $\mu_{x_m \rightarrow f_s}(x_m)$, we see that if a leaf node is
  * a variable node: the message that it sends along its one and only link is given by $\mu_{x \rightarrow f}(x) = 1$, as illustrated in Figure 8.49(a). 
  * factor node: from $\mu_{f_s \rightarrow x}(x)$ that the message sent should take the form $\mu_{f \rightarrow x}(x) = f(x)$, as illustrated in Figure 8.49(b)

* Now suppose we wish to find the marginals for every variable node in the graph.

**Naive way**
* This could be done by simply running the above algorithm afresh for each such node.
* However, this would be very wasteful as many of the required computations would be repeated. 

**Better way**
* We can obtain a much more efficient procedure by ‘overlaying’ these multiple message passing algorithms to obtain the general sum-product algorithm as follows.
* Arbitrarily pick any (variable or factor) node and designate it as the root.
* Propagate messages from the leaves to the root as before.
* At this point, the root node will have received messages from all of its neighbours. It can therefore send out messages to all of its neighbours.
* These in turn will then have received messages from all of their neighbours and so can send out messages along the links going away from the root, and so on. 
* In this way, messages are passed outwards from the root all the way to the leaves.
* By now, a message will have passed in both directions across every link in the graph, and every node will have received a message from all of its neighbours.
* Again a simple inductive argument can be used to verify the validity of this message passing protocol.
  * Every variable node will have received messages from all of its neighbours
  * So we can readily calculate the marginal distribution for every variable in the graph.
* The number of messages that have to be computed is given by twice the number of links in the graph and so involves only twice the computation involved in finding a single marginal. 
* By comparison, if we had run the sum-product algorithm separately for each node, the amount of computation would grow quadratically with the size of the graph.
* Note that this algorithm is in fact independent of which node was designated as the root,